# Transformaciones de variables explicativas


**Flujo**
1. **Batch de entrada**: Inputs (`image_frames`, `ego_history_xyz/rot`, `ego_future_xyz/rot`, etc.).
2. **De trayectoria (xyz, rot) a acciones**: conversión de `ego_future_xyz` y `ego_future_rot` a aceleración y curvatura (Unicycle accel-curvature).
3. **Transformaciones sobre acciones**: clipping, estandarización (z-score), discretización en bins y tokenización (DiscreteTrajectoryTokenizer).
4. **Reconstrucción inversa**: de tokens → acciones → trayectoria (unicycle).
5. **Ejemplo**: Batch hasta tokens y métricas de error (MAE, error en trayectoria).


## 0. Inputs

Se cuentan con los siguientes campos:

| Campo | Forma típica | Descripción |
|-------|--------------|-------------|
| `image_frames` | (N_cámaras, num_frames, 3, H, W) | Imágenes de cámaras |
| `camera_indices` | (N_cámaras,) | Índices de cámara |
| `ego_history_xyz` | (1, 1, num_history_steps, 3) | Posición ego en el pasado (ej. 16 steps) |
| `ego_history_rot` | (1, 1, num_history_steps, 3, 3) | Rotación ego en el pasado |
| **`ego_future_xyz`** | **(1, 1, num_future_steps, 3)** | **Target: posición ego futura (ej. 64 steps)** |
| **`ego_future_rot`** | **(1, 1, num_future_steps, 3, 3)** | **Target: rotación ego futura** |
| `relative_timestamps` | (N_cámaras, num_frames) | Timestamps relativos |
| `absolute_timestamps` | (N_cámaras, num_frames) | Timestamps absolutos |
| `t0_us` | escalar | Timestamp de referencia (µs) |
| `clip_id` | str | Identificador del clip |

Las **variables objetivo** que el modelo debe predecir (y sobre las que se aplican las transformaciones que justificamos aquí) se derivan de **`ego_future_xyz`** y **`ego_future_rot`**. En el config de Alpamayo, el espacio de acciones es **Unicycle accel-curvature** con 64 waypoints y `dt=0.1` s; por tanto, internamente la trayectoria futura (xyz, rot) se convierte a secuencias de **aceleración** y **curvatura**, y luego se tokenizan. Eso es lo que reconstruimos en este notebook.

In [1]:
import json
import os
import numpy as np


CFG_PATH = os.path.expanduser("~/Downloads/transform_config.json")
if not os.path.isfile(CFG_PATH):
    CFG_PATH = "transform_config.json"  # 
with open(CFG_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

list(cfg.keys())


['action_in_proj_cfg',
 'action_out_proj_cfg',
 'action_space_cfg',
 'add_special_tokens',
 'architectures',
 'attn_implementation',
 'diffusion_cfg',
 'dtype',
 'expert_cfg',
 'expert_non_causal_attention',
 'hist_traj_tokenizer_cfg',
 'keep_same_dtype',
 'max_pixels',
 'min_pixels',
 'model_dtype',
 'model_type',
 'tokens_per_future_traj',
 'tokens_per_history_traj',
 'traj_token_ids',
 'traj_token_start_idx',
 'traj_tokenizer_cfg',
 'traj_vocab_size',
 'transformers_version',
 'vlm_backend',
 'vocab_size']

## 1. ¿Qué aplicamos aqui?

Del config tomamos principalmente:
- `action_space_cfg` (dinámica y estadísticos)
- `traj_tokenizer_cfg` (rango y discretización)
- `n_waypoints`, `dt`
- `accel_bounds`, `curvature_bounds`, `mean`, `std`



In [2]:
action_space = cfg["action_space_cfg"]
tokenizer = cfg["traj_tokenizer_cfg"]

n_waypoints = action_space["n_waypoints"]
dt = action_space["dt"]

accel_bounds = action_space["accel_bounds"]
curv_bounds  = action_space["curvature_bounds"]

accel_mean = action_space["accel_mean"]
accel_std  = action_space["accel_std"]
curv_mean  = action_space["curvature_mean"]
curv_std   = action_space["curvature_std"]

dims_min = tokenizer["dims_min"]
dims_max = tokenizer["dims_max"]
num_bins = tokenizer["num_bins"]

(n_waypoints, dt, accel_bounds, curv_bounds, num_bins)


(64, 0.1, [-9.8, 9.8], [-0.33, 0.33], 3000)

### 1.1 De ego_future_xyz y ego_future_rot a (aceleración, curvatura)

El modelo no entrena directamente sobre posiciones futuras $(x,y,z)$ y matrices de rotación: usa el **espacio de acciones Unicycle** (aceleración $a_t$ y curvatura $\kappa_t$). Hay que obtener $(a, \kappa)$ a partir de la trayectoria.

**Supuestos (defendibles en tesis):**
- Trayectoria en marco **ENU** (East-North-Up); movimiento predominante en el plano $(x,y)$; la rotación es solo **yaw** (eje Z).
- De `ego_future_xyz` tomamos $(x_t, y_t)$ (o solo east/north). De `ego_future_rot` extraemos el yaw $\theta_t$ (p. ej. de la matriz 3×3 o vía atan2).
- Velocidad escalar y curvatura:
  - $v_t = \| (x_{t+1}-x_t,\, y_{t+1}-y_t) \| / \Delta t$
  - $\theta_t = \mathrm{yaw}(\mathrm{rot}_t)$
  - Curvatura: $\kappa_t = (\theta_{t+1} - \theta_t) / (v_t \Delta t + \epsilon)$ (cambio de heading por unidad de distancia).
  - Aceleración: $a_t = (v_{t+1} - v_t) / \Delta t$.

Con esto obtenemos secuencias `accel[T]` y `curvature[T]` (longitud $T = \texttt{num\_future\_steps} - 1$ o $T = 64$ si se rellenan bordes), que son las **variables objetivo** a las que luego se aplican clipping, estandarización y discretización.

In [3]:
def yaw_from_rotation_matrix(rot: np.ndarray) -> np.ndarray:
    """Extract yaw (rad) from a 3x3 rotation matrix (Z-axis rotation only). rot: (..., 3, 3)."""
    # R = Rz(yaw) => cos(yaw)=R[0,0], sin(yaw)=R[1,0]
    r = np.asarray(rot)
    if r.ndim == 2:
        return np.arctan2(r[1, 0], r[0, 0])
    return np.arctan2(r[..., 1, 0], r[..., 0, 0])


def trajectory_xyz_rot_to_accel_curvature(
    xyz: np.ndarray,
    rot: np.ndarray,
    dt: float,
    eps: float = 1e-9,
    v_min: float = 0.5,
    accel_bounds: tuple = (-9.8, 9.8),
    curvature_bounds: tuple = (-0.33, 0.33),
) -> tuple[np.ndarray, np.ndarray]:
    """
    Convert trajectory (xyz, rot) to acceleration and curvature (Unicycle model).
    v_min: minimum speed (m/s) for curvature computation to avoid huge kappa when v~0.
    accel_bounds, curvature_bounds: physical clipping limits (m/s^2 and rad/m).
    """
    xyz = np.squeeze(np.asarray(xyz, dtype=np.float64))
    rot = np.squeeze(np.asarray(rot, dtype=np.float64))
    if xyz.ndim == 3:
        xyz = xyz.reshape(-1, 3)
    if rot.ndim == 4:
        rot = rot.reshape(-1, 3, 3)
    N = xyz.shape[0]
    assert rot.shape[0] == N

    x, y = xyz[:, 0], xyz[:, 1]
    theta = yaw_from_rotation_matrix(rot)

    dx = np.diff(x)
    dy = np.diff(y)
    dtheta = np.diff(theta)

    v_mid = np.sqrt(dx**2 + dy**2) / dt  # (N-1,)
    v_safe = np.maximum(v_mid, v_min)  # avoid huge kappa when v~0

    v0 = np.concatenate([[v_mid[0]], v_mid])
    v1 = np.concatenate([v_mid, [v_mid[-1]]])
    accel = (v1[1:] - v0[:-1]) / dt

    # Curvature kappa = dtheta/(v*dt) in rad/m; v_safe prevents division by ~0
    curvature = dtheta / (v_safe * dt + eps)

    accel = np.clip(accel, accel_bounds[0], accel_bounds[1])
    curvature = np.clip(curvature, curvature_bounds[0], curvature_bounds[1])
    return accel.astype(np.float64), curvature.astype(np.float64)

## 2. Transformaciones

### 2.1 Clipping por límites físicos
Para cada paso `t`:

$$
a_t \leftarrow \mathrm{clip}(a_t, a_{\min}, a_{\max})
$$
$$
\kappa_t \leftarrow \mathrm{clip}(\kappa_t, \kappa_{\min}, \kappa_{\max})
$$

Esto evita targets físicamente imposibles y estabiliza entrenamiento.

### 2.2 Estandarización (z-score)
Usando media y desviación estándar del dataset:

$$
\hat{a}_t = \frac{a_t - \mu_a}{\sigma_a}
\qquad
\hat{\kappa}_t = \frac{\kappa_t - \mu_\kappa}{\sigma_\kappa}
$$

Esto pone magnitudes en escala comparable, mejora condicionamiento numérico, y ayuda a que un MLP/Transformer aprenda más fácil.

### 2.3 Discretización a bins (para tokenizar)
Si el tokenizer discretiza cada dimensión a `num_bins` en $[m, M]$, una forma estándar es:

$$
b = \left\lfloor \frac{(x - m)}{(M - m)} \cdot (B-1) \right\rceil
$$
donde:
- $x$ es el valor continuo (p. ej. $\hat{a}$ o $\hat{\kappa}$)
- $B$ es `num_bins`
- $\lfloor \cdot \rceil$ redondeo al entero más cercano
- luego se hace `clip(b, 0, B-1)`

Y la inversión:

$$
x \approx m + \frac{b}{B-1}(M-m)
$$

Esto permite convertir targets continuos en **tokens discretos**, compatible con modelos tipo LLM.

### 2.4 Unicycle model
Si controlas con velocidad $v$ y curvatura $\kappa$:

$$
\theta_{t+1} = \theta_t + v_t \kappa_t \Delta t
$$
$$
x_{t+1} = x_t + v_t \cos(\theta_t) \Delta t
$$
$$
y_{t+1} = y_t + v_t \sin(\theta_t) \Delta t
$$

Si el control es aceleración $a$, entonces:
$$
v_{t+1} = v_t + a_t \Delta t
$$



In [4]:
def clip_bounds(x, lo, hi):
    return np.minimum(np.maximum(x, lo), hi)

def standardize(x, mean, std, eps=1e-12):
    return (x - mean) / (std + eps)

def unstandardize(z, mean, std):
    return z * std + mean

def quantize_to_bins(x, xmin, xmax, num_bins):
    """Map continuous x to integer bins in [0, num_bins-1]."""
    x = clip_bounds(x, xmin, xmax)
    # scale to [0, num_bins-1]
    b = np.rint((x - xmin) / (xmax - xmin) * (num_bins - 1)).astype(np.int64)
    return clip_bounds(b, 0, num_bins - 1)

def dequantize_from_bins(b, xmin, xmax, num_bins):
    b = clip_bounds(b, 0, num_bins - 1).astype(np.float64)
    return xmin + (b / (num_bins - 1)) * (xmax - xmin)


## 3. Targets continuos a tokens

Aquí construimos una función que:
1. clip: `accel` y `curvature`
2. estandarizar con (mean, std)
3. discretizar a bins dentro de `dims_min/dims_max`
4. producir un par de tokens por timestep (a, kappa)

`dims_min=[-10,-10]` y `dims_max=[10,10]`. Eso sugiere que la discretización opera sobre una representación ya acotada. Una interpretación razonable y defendible es que discretizas los valores **estandarizados** (z-scores) dentro de [-10, 10].


In [5]:
def targets_to_tokens(accel, curv, *, use_standardize=True):
    """accel, curv: arrays shape [T]"""
    accel = np.asarray(accel, dtype=np.float64)
    curv  = np.asarray(curv,  dtype=np.float64)
    assert accel.shape == curv.shape

    # 1) clip in physical bounds
    a = clip_bounds(accel, accel_bounds[0], accel_bounds[1])
    k = clip_bounds(curv,  curv_bounds[0],  curv_bounds[1])

    # 2) standardize 
    if use_standardize:
        a_z = standardize(a, accel_mean, accel_std)
        k_z = standardize(k, curv_mean, curv_std)
    else:
        a_z, k_z = a, k

    # 3) quantize to bins in dims_min/dims_max
    a_bin = quantize_to_bins(a_z, dims_min[0], dims_max[0], num_bins)
    k_bin = quantize_to_bins(k_z, dims_min[1], dims_max[1], num_bins)

    # shape [T, 2]
    return np.stack([a_bin, k_bin], axis=-1)

def tokens_to_targets(tokens, *, use_standardize=True):
    tokens = np.asarray(tokens, dtype=np.int64)
    assert tokens.ndim == 2 and tokens.shape[1] == 2

    a_bin = tokens[:, 0]
    k_bin = tokens[:, 1]

    # 1) dequantize back to continuous in dims range
    a_z = dequantize_from_bins(a_bin, dims_min[0], dims_max[0], num_bins)
    k_z = dequantize_from_bins(k_bin, dims_min[1], dims_max[1], num_bins)

    # 2) unstandardize 
    if use_standardize:
        a = unstandardize(a_z, accel_mean, accel_std)
        k = unstandardize(k_z, curv_mean, curv_std)
    else:
        a, k = a_z, k_z

    # 3) clip to physical bounds again for safety
    a = clip_bounds(a, accel_bounds[0], accel_bounds[1])
    k = clip_bounds(k, curv_bounds[0],  curv_bounds[1])
    
    return np.stack([a, k], axis=-1)


## 4. Reconstrucción de trayectoria con modelo unicycle

Integración:
- $v_{t+1} = \mathrm{clip}(v_t + a_t \, dt,\, 0,\, v_{\max})$ (asumimos no reversa)
- $\theta_{t+1} = \theta_t + v_t \, \kappa_t \, dt$
- $(x,y)$ por Euler


In [ ]:
# Example: filter variable names containing 'n'
variables_con_n = [n for n in dir() if 'n' in n]
print(variables_con_n)

In [16]:
def rollout_unicycle(accel, kappa, *, dt=0.1, v0=0.0, v_max=40.0, x0=0.0, y0=0.0, th0=0.0):
    accel = np.asarray(accel, dtype=np.float64)
    kappa = np.asarray(kappa, dtype=np.float64)
    T = accel.shape[0]

    x = np.zeros(T+1); y = np.zeros(T+1); th = np.zeros(T+1); v = np.zeros(T+1)
    x[0], y[0], th[0], v[0] = x0, y0, th0, v0

    for i in range(T):
        v[i+1] = clip_bounds(v[i] + accel[i]*dt, 0.0, v_max)
        th[i+1] = th[i] + v[i]*kappa[i]*dt
        x[i+1]  = x[i] + v[i]*np.cos(th[i])*dt
        y[i+1]  = y[i] + v[i]*np.sin(th[i])*dt
    return x, y, th, v

T = n_waypoints if 'n_waypoints' in dir() else 64
dt_step = dt if 'dt' in dir() else 0.1
t = np.arange(T) * dt_step
accel = accel_mean * np.sin(2 * np.pi * 0.2 * t)
curv = curv_mean * np.sin(2 * np.pi * 0.1 * t)
tokens = targets_to_tokens(accel, curv, use_standardize=True)
targets_rec = tokens_to_targets(tokens, use_standardize=True)
accel_rec = targets_rec[:, 0]
curv_rec = targets_rec[:, 1]

# Original vs reconstructed trajectory
x0, y0, th0, v0 = 0.0, 0.0, 0.0, 5.0
x, y, th, v = rollout_unicycle(accel, curv, dt=dt_step, v0=v0, x0=x0, y0=y0, th0=th0)
x_r, y_r, th_r, v_r = rollout_unicycle(accel_rec, curv_rec, dt=dt_step, v0=v0, x0=x0, y0=y0, th0=th0)
traj_err = np.mean(np.sqrt((x-x_r)**2 + (y-y_r)**2))
traj_err


0.004173317062556908

### 4.1 Pipeline completo: de batch (ego_future_xyz / ego_future_rot) a tokens

Aplicamos **todas** las transformaciones que reciben las variables objetivo del modelo, partiendo del formato de batch:

1. **Entrada**: `ego_future_xyz` (1, 1, 64, 3), `ego_future_rot` (1, 1, 64, 3, 3).
2. **Trayectoria a acciones**: `trajectory_xyz_rot_to_accel_curvature` - `accel` (63,), `curvature` (63,).
3. **Acciones a tokens**: clip, estandarización, discretización - matriz de tokens (63, 2) (o 64 si se rellena; el config usa 64 waypoints).
4. **Inversa**: tokens a acciones a trayectoria unicycle; comparamos con la original para reportar error.


### 4.2 Ejemplo con un clip muestra: 

Cargamos un clip guardado del conjunto de train (De un dict de tensores). El archivo `.pt` suele contener las mismas keys que el batch: `image_frames`, `ego_history_xyz`, `ego_future_xyz`, `ego_future_rot`, `t0_us`, `clip_id`, etc. Aquí usamos **solo las variables objetivo** (`ego_future_xyz`, `ego_future_rot`) para aplicar el pipeline y reportar MAE con datos reales.

In [17]:
import torch

CLIP_PT_PATH = "clip_sample.pt"
if not os.path.isfile(CLIP_PT_PATH):
    CLIP_PT_PATH = os.path.join(os.path.dirname(os.path.abspath(".")), "clip_sample.pt")

clip_data = torch.load(CLIP_PT_PATH, map_location="cpu", weights_only=False)

print("Keys en clip_7.pt:", list(clip_data.keys()))
for k in clip_data:
    v = clip_data[k]
    if torch.is_tensor(v):
        print(f"  {k}: shape {tuple(v.shape)}, dtype {v.dtype}")
    else:
        print(f"  {k}: {type(v).__name__} = {v}")

t0_us = clip_data.get("t0_us", None)
clip_id = clip_data.get("clip_id", None)
print("\nt0_us:", t0_us, "  clip_id:", clip_id)



Keys en clip_7.pt: ['image_frames', 'camera_indices', 'ego_history_xyz', 'ego_history_rot', 'ego_future_xyz', 'ego_future_rot', 'relative_timestamps', 'absolute_timestamps', 't0_us', 'clip_id']
  image_frames: shape (4, 4, 3, 1080, 1920), dtype torch.uint8
  camera_indices: shape (4,), dtype torch.int64
  ego_history_xyz: shape (1, 1, 16, 3), dtype torch.float32
  ego_history_rot: shape (1, 1, 16, 3, 3), dtype torch.float32
  ego_future_xyz: shape (1, 1, 64, 3), dtype torch.float32
  ego_future_rot: shape (1, 1, 64, 3, 3), dtype torch.float32
  relative_timestamps: shape (4, 4), dtype torch.float32
  absolute_timestamps: shape (4, 4), dtype torch.int64
  t0_us: int = 1760117223655282
  clip_id: str = f91ecb1d-4039-4a81-877e-4c67c000d138

t0_us: 1760117223655282   clip_id: f91ecb1d-4039-4a81-877e-4c67c000d138


In [19]:

ego_future_xyz_clip = clip_data["ego_future_xyz"]
ego_future_rot_clip = clip_data["ego_future_rot"]
if torch.is_tensor(ego_future_xyz_clip):
    ego_future_xyz_clip = ego_future_xyz_clip.numpy()
if torch.is_tensor(ego_future_rot_clip):
    ego_future_rot_clip = ego_future_rot_clip.numpy()

# 1) Trajectory to actions (accel, curvature) from clip_sample
accel_clip, curv_clip = trajectory_xyz_rot_to_accel_curvature(
    ego_future_xyz_clip, ego_future_rot_clip, dt=dt
)
# Pad to n_waypoints=64 if needed
if len(accel_clip) == n_waypoints - 1:
    accel_clip = np.concatenate([accel_clip, [accel_clip[-1]]])
    curv_clip = np.concatenate([curv_clip, [curv_clip[-1]]])

# 2) Actions -> tokens -> actions (targets as (64, 2))
tokens_from_clip = targets_to_tokens(accel_clip, curv_clip, use_standardize=True)
targets_rec_clip = tokens_to_targets(tokens_from_clip, use_standardize=True)  # (64, 2)

# 3) Metrics: MAE of reconstructed actions vs original
mae_a_clip = np.mean(np.abs(accel_clip - targets_rec_clip[:, 0]))
mae_k_clip = np.mean(np.abs(curv_clip - targets_rec_clip[:, 1]))
print("Pipeline clip_sample - tokens - acciones (MAE):", "accel =", mae_a_clip, "curv =", mae_k_clip)
print("tokens_from_clip:", tokens_from_clip.shape, " targets reconstruidos:", targets_rec_clip.shape)

Pipeline clip_sample - tokens - acciones (MAE): accel = 1.1903538532351297 curv = 0.022124923116007236
tokens_from_clip: (64, 2)  targets reconstruidos: (64, 2)


## 5. Conclusión

- **Batch de entrada**: el modelo recibe `ego_future_xyz` y `ego_future_rot` como variables objetivo (trayectoria futura en marco ego). Se justifica la conversión a espacio de acciones (aceleración y curvatura) mediante derivación numérica de velocidad y curvatura a partir de posiciones y yaw (supuesto: movimiento en plano, rotación solo en Z).
- **Bounds físico**: bounds de aceleración y curvatura reflejan restricciones del vehículo.
- **Clipping**: evita targets fuera de distribución y estabiliza entrenamiento.
- **Estandarización**: reduce escala y mejora el acondicionamiento numérico; alinea magnitudes.
- **Discretización**: convierte control continuo en una secuencia de símbolos; esto habilita entrenamiento estilo language modeling.
- **Error de cuantización**: medible como MAE en acciones y error en trayectoria; en el ejemplo reportamos ambos.
- **Inversión**: demostrar que puedes volver de tokens a acciones y a trayectoria cierra el ciclo y hace la metodología reproducible.

